In [2]:
import pandas as pd
import re
import wikipedia
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

In [1]:
# https://medium.com/@Alexander_H/scraping-wikipedia-with-python-8000fc9c9e6c

In [6]:
networks = pd.read_csv('./Untitled spreadsheet - Sheet1.csv')
net_list = list(networks.values.flatten())

In [8]:
net_list[:5]

['https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_American_Broadcasting_Company',
 'https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Adult_Swim',
 'https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_A%26E',
 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Amazon',
 'https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_AMC']

In [3]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_American_Broadcasting_Company')
b = BeautifulSoup(html.text, 'lxml')
links = []

for i in b.find_all(name = 'li'):
    for link in i.find_all('a', href=True):
        links.append(link['href'])
links = links[77:80]
abc_links = ['https://en.wikipedia.org' + i for i in links]

len(abc_links)

3

In [9]:
tv_titles = []
tv_links = []


for genre in net_list:
    print(f'Collecting shows from {genre}')
    html = requests.get(genre)
    b = BeautifulSoup(html.text, 'lxml')
    # get to the table on the page
    for i in b.find_all(name='table', class_='wikitable'):
        # get to the row of each show
        for j in i.find_all(name='tr'):
            #get just the title cell for each row.
            # contains the title and the URL
            for k in j.find_all(name='i'):
                # get within that cell to just get the words
                for link in k.find_all('a', href=True):
                    # get the title and add to the list
                    tv_titles.append(link['title'])
                    # get the link and add to that list
                    tv_links.append(link['href'])
    #be a conscientious scraper and pause between scrapes
    time.sleep(1)
print(f'Number of TV Links Collected: {len(tv_links)}')
print(f'Number of TV Titles Collected: {len(tv_titles)}')
# remove film links that don't have a description page on Wikipedia
new_tv_links = [i for i in tv_links if 'redlink' not in i]
# same goes for titles
new_tv_titles = [i for i in tv_titles if '(page does not exist)' not in i]
print(f'Number of TV Links with Wikipedia Pages: {len(new_tv_links)}')
print(f'Number of TV Titles with Wikipedia Pages: {len(new_tv_titles)}')
#use this list to fetch from the API
title_links = list(zip(new_tv_titles, new_tv_links))

KeyError: 'title'

In [4]:
new_tv_titles

['The Simpsons', 'American Dad!', 'Criminal Minds']

In [5]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Fox')
b = BeautifulSoup(html.text, 'lxml')
links = []

for i in b.find_all(name = 'li'):
    for link in i.find_all('a', href=True):
        links.append(link['href'])
links = links[57:]
fox_links = ['https://en.wikipedia.org' + i for i in links]

len(fox_links)
# Need to remove cites, external links, related links, List links

1351

In [6]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_NBC')
b = BeautifulSoup(html.text, 'lxml')
links = []

for i in b.find_all(name = 'li'):
    for link in i.find_all('a', href=True):
        links.append(link['href'])
links = links[7:]
nbc_links = ['https://en.wikipedia.org' + i for i in links]

len(nbc_links)

876

In [7]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_CBS')
b = BeautifulSoup(html.text, 'lxml')
links = []

for i in b.find_all(name = 'li'):
    for link in i.find_all('a', href=True):
        links.append(link['href'])
links = links[7:]
cbs_links = ['https://en.wikipedia.org' + i for i in links]

len(cbs_links)

1107

Currently have a list of wikis
- Need to extract the show name (can be done simultaneously via the href title?)
- Need to then take that list of names and use the wiki API to query the tables?? 
- NEED to grab: viewership information, logline
- WANT to grab: writer, air date, rating, director, ??

TVDV or Trakt API

- Take the list of names from the wiki to return IDs
- Use the IDs to grab the metadata
- Combine the metadata with the wiki information in a dataframe
- NEED: number of episodes, number of seasons, air date, producer, network, ?

https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=jsonfm&formatversion=2&titles=List_of_Modern_Family_episodes

https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=jsonfm&formatversion=2&titles=Grey%27s_Anatomy_(season_1)

two formats - List_of_SHOW_NAME_episodes & SHOW_NAME_(season_#)

Sample API for wikipedia from https://www.mediawiki.org/wiki/API:Query#Sample_query